From the beginning a clear concept: "everything is a field". We think that constant is just a special case of field which does not vary in temporal and spatial direction. Fields can vary in spatial direction (constant or variable) and in temporal direction (time variant and in-variant).

From here we can deduct for types of (discrete) fields:

- discrete, constant, time invariant (DCTI)
- discrete, variable, time invariant (DVTI)
- discrete, constant, time variant (DCTV)
- discrete, variable, time variant (DVTV)

Field itself can be anything. However, usually either scalar, vector or tensor (matrix).

# Creating fields

For discrete fields that are varying in spatial direction, value for each discrete point is defined using NTuple. The order of points is implicitly assumed to be same than node ordering in ABAQUS. Usual use case for variable field is that the field is interpolated to the element volume using interpolation polynomials, i.e. $u(\xi) = u_i N_i(\xi)$, where $N_i$ is the basis function for that node and $u_i$ is the discrete value.

For example, `(1, 2, 3, 4)` is a scalar field having length of 4 and `([1,2],[2,3],[3,4],[4,5])` is a vector field having length of 4.

For fields that are varying in temporal direction, `time => value` syntax is used. The first item in pair is time and second item is value attached to that time. For example, `0.0 => 1.0` is a time-dependent scalar field having value 1.0 at time 0.0.

The most simple field is a field that is constant in both time and spatial direction. Discrete, constant, time invariant (DCTI):

In [1]:
a = DCTI(1.0)

LoadError: [91mUndefVarError: DCTI not defined[39m

Then we have discrete, variable, time invariant fields (DVTI). Note the use of `NTuple` when defining field.

In [24]:
b = DVTI( (1.0, 2.0) )

LoadError: [91mUndefVarError: DVTI not defined[39m

Discrete, constant, time variant field (DCTV) is constant in spatial direction but can vary in temporal direction. Here, `=>` syntax is used. New values can be added to field using `update!`. If there already exists value for that particular time, it will be overridden. It is assumed that content of field in time direction is monotonically increasing, i.e. $t_{i-1} < t_i < t_{i+1}$. For the sake of clarity let's also mention that `update!` works for time invariant fields as well if content needs to be updated.

In [22]:
c = DCTV(0.0 => 1.0, 1.0 => 2.0)
update!(c, 2.0 => 3.0)

LoadError: [91mUndefVarError: DCTV not defined[39m

Discrete, variable, time variant (DVTV) field is the most general one, allowing values of field to vary in both spatial and time direction.

In [23]:
d = DVTV(0.0 => (1.0, 2.0), 1.0 => (2.0, 3.0))
update!(d, 2.0 => (3.0 4.0))

LoadError: [91mUndefVarError: DVTV not defined[39m

In examples above, all fields was scalar fields. Defining vector or tensor fields goes in the same spirit. Only difference is that now we define vectors and tensors, not a single scalar value. They can vary in spatial and time direction in the same way than scalar fields.

In [6]:
a = DCTI([1.0, 2.0])
b = DVTI(([1.0, 2.0], [2.0, 3.0]))
c = DCTV(0.0 => [1.0, 2.0], 1.0 => [2.0, 3.0])
d = DVTV(0.0 => ([1.0, 2.0], [2.0, 3.0]), 1.0 => ([2.0, 3.0], [3.0, 4.0]))

LoadError: [91mUndefVarError: DCTI not defined[39m

# Querying fields

Accessing fields is done using only one command: `interpolate`. For time varying fields, one can interpolate in time direction:

In [14]:
interpolate(c, 0.5)

LoadError: [91mUndefVarError: interpolate not defined[39m

In [15]:
interpolate(d, 0.5)

LoadError: [91mUndefVarError: interpolate not defined[39m

One should be always able to interpolate in time direction, even if field is time invariant, to get trivial solution:

In [16]:
interpolate(a, 0.5), interpolate(b, 0.5),
interpolate(c, 0.5), interpolate(d, 0.5)

LoadError: [91mUndefVarError: interpolate not defined[39m

For spatially varying fields, one can access to ith element using getindex:

In [17]:
getindex(a, 1), getindex(b, 1), getindex(b, 2)

LoadError: [91mUndefVarError: a not defined[39m

First time interpolation, then spatial lookup, i.e.

In [18]:
getindex(interpolate(d, 0.5), 1)

LoadError: [91mUndefVarError: interpolate not defined[39m

Shortcuts: functor is interpolating in time direction and [] for getindex, i.e.

In [20]:
r1 = getindex(interpolate(d, 0.5), 1)
r2 = d(0.5)[1]
isapprox(r1, r2)

LoadError: [91mUndefVarError: interpolate not defined[39m

Then we need to have some function to combine discrete fields with (already evaluated) basis functions.

In [21]:
N = [1.0 2.0 3.0 4.0] # evaluated basis functions
t = 1.0 # time
results = interpolate(d, t, N)
interpolate!(results, d, t, N)

LoadError: [91mUndefVarError: interpolate not defined[39m

# Continuous fields

Then we have continuous fields which may be useful when defining analytical boundary conditions.

# Dictionary fields

One can also put dictionary inside field. This can be used to define global "problem wide" fields.

# Using `Field`

One can use a single constructor called `Field` to create all kind of fields. Type of field is inspected from data type.

In [25]:
typeof(Field(1.0))

LoadError: [91mUndefVarError: Field not defined[39m

In [26]:
typeof(Field((1.0, 2.0)))

LoadError: [91mUndefVarError: Field not defined[39m

In [27]:
typeof(Field(0.0 => 1.0))

LoadError: [91mUndefVarError: Field not defined[39m

In [28]:
typeof(Field(0.0 => (1.0, 2.0), 1.0 => (2.0, 3.0)))

LoadError: [91mUndefVarError: Field not defined[39m